
# LLM for outputting a user-friendly message

Use the trained YAMNet+context model to produce predictions and ask a Groq LLM to turn them into clear guidance for end users.



### Loading model, data + encoder

Mount Drive and load the saved classifier, label encoder, and test arrays needed for inference.


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import joblib
import json

path = 'PATH/TO/YOUR/FILES/'

# Load the trained multimodal classifier
model = load_model(path + '/yamnet_model.keras')

# Load label encoder to turn indices into human-readable classes
le = joblib.load(path + '/label_encoder.plk')

# Cached test splits to reuse during inference demos
X_audio_test = np.load(path + '/X_audio_test.npy')
X_context_test = np.load(path + '/X_context_test.npy')


Mounted at /content/drive



### Install Groq client (Colab runtime)

Install the SDK in the runtime so we can call the hosted LLM.


In [3]:
!pip install groq  # One-time install per runtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.5 MB/s eta 0:00:00



### Configure Groq client

Initialize the client with your API key (ideally set as an environment variable instead of hard-coding it).


In [ ]:
from groq import Groq

# Replace the string with the API key obtained from the Groq Cloud console
client = Groq(api_key="API_KEY_STRING")



### Prompt builder for structured explanations

Build a JSON-only prompt that keeps each predicted issue independent and outputs user-friendly advice.


In [42]:
prompt = """
You are an AI automotive mechanic assistant. You MUST output ONLY valid JSON with no Markdown, no backticks, and no extra text.

You will receive a small set of predicted problems (top-1 or top-2). You must only return JSON objects for the problems provided.

STRICT OUTPUT RULES:
1. If only top-1 is provided, output EXACTLY one JSON object inside a JSON array.
2. If top-2 is provided, output EXACTLY two JSON objects.
3. NEVER output more problems than those provided.

PROBLEM LABELS (allowed values):
- "Worn Out Brakes"
- "Normal Brakes"
- "Serpentine Belt"
- "Car Knocking"
- "Normal Engine Idle"
- "Power Steering"
- "Low Oil"
- "Bad Ignition"
- "Dead Battery"
- "Normal Engine Sound"
- "Normal Engine Startup"

SPECIAL RULE FOR "NORMAL" PROBLEMS:
If the problem contains "Normal":
- Explain that the sound/behavior is normal.
- Provide light monitoring advice.

CRITICAL GLOBAL RULE (APPLIES TO EVERY FIELD OF THE JSON):
You MUST NOT mention, imply, or describe any cause that corresponds to ANY item from the official problem list.

This restriction applies to:
- explanation
- possible_causes
- recommended_actions
- what_to_tell_the_mechanic
- advice

FORBIDDEN TERMS (DO NOT APPEAR ANYWHERE IN THE OUTPUT):
- anything equivalent to any problem label

If a concept belongs to one of the diagnostic labels above, DO NOT USE IT anywhere in the JSON.

Make clear explanations.

OUTPUT FORMAT FOR EACH PROBLEM:
{
  "problem": <string>,
  "probability": <float>,
  "severity": <"low" | "moderate" | "high">,
  "explanation": <string WITHOUT any forbidden terms>,
  "possible_causes": <list WITHOUT any forbidden terms>,
  "recommended_actions": <list WITHOUT any forbidden terms>,
  "what_to_tell_the_mechanic": <string WITHOUT any forbidden terms>,
  "advice": <string WITHOUT any forbidden terms>
}

Your final output MUST be ONLY a valid JSON array with the above objects, nothing else.

"""


def generate_message_json(top1_problem, top1_prob, top2_problem=None, top2_prob=None):
    """Create a JSON-only prompt for the LLM and parse its response."""

    # Build user message dynamically
    if top2_problem is None:
        user_payload = {
            "top_1_problem": top1_problem,
            "top_1_probability": top1_prob
        }
    else:
        user_payload = {
            "top_1_problem": top1_problem,
            "top_1_probability": top1_prob,
            "top_2_problem": top2_problem,
            "top_2_probability": top2_prob
        }

    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": json.dumps(user_payload)}
    ]

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=messages,
        temperature=0.2,
        max_tokens=500
    )

    raw_output = response.choices[0].message.content.strip()

    # JSON should be clean, but just in case we remove backticks
    cleaned_output = raw_output.replace("```json", "").replace("```", "").strip()

    try:
        decoded = json.loads(cleaned_output)
    except json.JSONDecodeError:
        decoded = {"error": "Invalid JSON", "raw": raw_output}

    return decoded



### Run model inference and request explanation

Call the trained model to get class probabilities, then format a prompt for the LLM based on confidence levels.


In [6]:
def full_diagnosis(i):
    """Run inference on the test split, decide whether to ask for top-1 or top-2, and fetch the JSON explanation."""

    # Prepare model inputs
    emb = X_audio_test[i].reshape(1, -1)
    ctx = np.array([[int(X_context_test[i])]], dtype=np.int32)

    # Predictions
    probs = model.predict([emb, ctx], verbose=0)[0]

    # Top 3 ordered
    top_idx = probs.argsort()[-3:][::-1]

    top1_class = le.classes_[top_idx[0]]
    top1_prob = float(probs[top_idx[0]])

    # Case 1 → Top-1 only
    if top1_prob >= 0.70:
        explanation_json = generate_message_json(
            top1_problem=top1_class,
            top1_prob=top1_prob
        )
        return {
            "mode": "single",
            "prediction": top1_class,
            "confidence": top1_prob,
            "explanation_json": explanation_json
        }

    # Case 2 → Top-2
    top2_class = le.classes_[top_idx[1]]
    top2_prob = float(probs[top_idx[1]])

    explanation_json = generate_message_json(
        top1_problem=top1_class,
        top1_prob=top1_prob,
        top2_problem=top2_class,
        top2_prob=top2_prob
    )

    return {
        "mode": "top2",
        "predictions": [
            (top1_class, top1_prob),
            (top2_class, top2_prob)
        ],
        "explanation_json": explanation_json
    }



### Example calls

Sample invocations on different test indices. Adjust the index to explore other audio clips.


In [ ]:
result = full_diagnosis(150)
result


{'mode': 'single',
 'prediction': np.str_('car_knocking'),
 'confidence': 0.9947447776794434,
 'explanation_json': [{'problem': 'Car Knocking',
   'probability': 0.9947447776794434,
   'severity': 'high',
   'explanation': 'The engine is producing a knocking or tapping sound, which can be caused by various factors.',
   'possible_causes': ['Worn engine bearings',
    'Low engine oil levels',
    'Faulty engine mounts'],
   'recommended_actions': ['Check engine oil levels and top off as needed',
    'Inspect engine mounts for wear or damage',
    'Consider a professional engine inspection'],
   'what_to_tell_the_mechanic': "The car is making a knocking sound, and I'm concerned it might be a serious issue.",
   'advice': 'Monitor the sound and report any changes to the mechanic.'}]}

In [44]:
result = full_diagnosis(291)
result


{'mode': 'single',
 'prediction': np.str_('dead_battery'),
 'confidence': 0.9993599057197571,
 'explanation_json': [{'problem': 'Dead Battery',
   'probability': 0.9993599057197571,
   'severity': 'high',
   'explanation': 'The battery may be drained due to an electrical issue.',
   'possible_causes': ['Electrical system malfunction',
    'Faulty charging system'],
   'recommended_actions': ['Have the battery tested and replaced if necessary',
    'Check the electrical system for any issues'],
   'what_to_tell_the_mechanic': "The vehicle's battery may be dead, and it needs to be checked and possibly replaced.",
   'advice': 'Have the vehicle towed to a repair shop and have the battery checked as soon as possible.'}]}


The `explanation_json` returned by `full_diagnosis` can be fed directly into a UI or voice assistant to give drivers friendly, actionable guidance.
